# Preprocessing data and extracting new features

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

In [65]:
number = 6
df = pd.read_csv('yellow_tripdata_2016-0{}.csv'.format(number), header=0)

### Drop all the odd cells and normilize time

In [66]:
df = df[df['tpep_dropoff_datetime'] != df['tpep_pickup_datetime']]
df = df[df['passenger_count'] != 0]
df = df[df['trip_distance'] != 0]
df = df[df['pickup_longitude'] >= -74.25559]
df = df[df['pickup_longitude'] <= -73.70001]
df = df[df['pickup_latitude'] >= 40.49612]
df = df[df['pickup_latitude'] <= 40.91553]
df['tpep_pickup_datetime'] =  pd.to_datetime(df['tpep_pickup_datetime'], format='%Y%m%d %H:%M:%S.%f')
df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].map(lambda x: x.replace(second=0))
df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].map(lambda x: x.replace(minute=0))
df['tpep_dropoff_datetime'] =  pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y%m%d %H:%M:%S.%f')
df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].map(lambda x: x.replace(second=0))
df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].map(lambda x: x.replace(minute=0))

In [67]:
dates = pd.DatetimeIndex(df['tpep_pickup_datetime'].values)
min_date = min(dates)

### Trips to cell feature

In [68]:
trips_to_cell = pd.read_csv('regions.csv', sep=';', header=0)
trips_to_cell.drop(['west', 'east', 'north', 'south'], axis=1, inplace=True)
for i in range(max(dates.day)*24):
    cur_datetime = pd.to_datetime(min_date) + np.timedelta64(i, 'h')
    df_cut_datetime = df[df['tpep_pickup_datetime'] == cur_datetime]
    ret = stats.binned_statistic_2d(df_cut_datetime['dropoff_longitude'],  df_cut_datetime['dropoff_latitude'],
                      None,  statistic='count',
                      bins=[np.linspace(-74.25559, -73.70001, 51) , np.linspace(40.49612, 40.91553, 51)])
    trips_to_cell[(str(cur_datetime)[:10] + '(hour: {})'.format(i%24))] = ret.statistic.reshape(2500)
trips_to_cell.to_csv('trips_to_region_{}.csv'.format(number), index=False)

C:\Users\ptimo\Anaconda3\lib\site-packages\scipy\stats\_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


## Amount of calls in the neighbours' cells

Let's use next implimentation to count mean value of calls in the closest cells for i-th cell

<img src="cell_implimentation.PNG">

Let's count calls from each cell (as we did before)

In [69]:
%%time
trips_by_regions = pd.read_csv('regions.csv', sep=';', header=0)
trips_by_regions.drop(['west', 'east', 'north', 'south'], axis=1, inplace=True)
for i in range(max(dates.day)*24):
    cur_datetime = pd.to_datetime(min_date) + np.timedelta64(i, 'h')
    df_cut_datetime = df[df['tpep_pickup_datetime'] == cur_datetime]
    ret = stats.binned_statistic_2d( df_cut_datetime['pickup_longitude'],  df_cut_datetime['pickup_latitude'],
                      None,  statistic='count',
                      bins=[np.linspace(-74.25559, -73.70001, 51) , np.linspace(40.49612, 40.91553, 51)])
    trips_by_regions[(str(cur_datetime)[:10] + '(hour: {})'.format(i%24))] = ret.statistic.reshape(2500)
trips_by_regions.to_csv('trips_in_region_{}.csv'.format(number), index=False)

C:\Users\ptimo\Anaconda3\lib\site-packages\scipy\stats\_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


Wall time: 43.4 s


In [63]:
active_cells = (1076, 1126, 1127, 1128, 1129, 1132, 1180, 1181, 1230, 1231, 1232, 1233, 1234, 1282,
 1283, 1339, 1383, 1431, 1684, 1734, 2069, 2119, 1075, 1125, 1221, 1227, 1272, 1326, 1331, 1382, 1434, 1441, 1480, 1482, 1483, 1530,
 1532, 1533, 1580, 1630, 1733, 1783, 2068, 2118, 2168, 1173, 1174, 1175, 1176, 1183, 1225, 1278, 1388, 1389, 1390, 1436, 1437, 1438, 1439,
 1442, 1130, 1131, 1172, 1177, 1178, 1179, 1222, 1223, 1224, 1228, 1229, 1273, 1274, 1279,
 1327, 1376, 1377, 1378, 1380, 1426, 1077, 1182, 1184, 1235, 1280, 1281, 1284, 1285, 1286, 1287, 1332, 1333, 1334, 1335,
 1336, 1337, 1338, 1384, 1385, 1386, 1387, 1435)

In [70]:
%%time
trips_by_regions_neighbours = pd.read_csv('regions.csv', sep=';', header=0)
trips_by_regions_neighbours = trips_by_regions_neighbours[trips_by_regions_neighbours['region'].isin(active_cells)]
trips_by_regions_neighbours.drop(['west', 'east', 'north', 'south'], axis=1, inplace=True)
row = 50
for i in range(max(dates.day)*24):
    region_mean = []
    cur_datetime = pd.to_datetime(min_date) + np.timedelta64(i, 'h')
    df_cut_datetime = df[df['tpep_pickup_datetime'] == cur_datetime]
    cur_time_column = str(cur_datetime)[:10] + '(hour: {})'.format(i%24)
    for j in active_cells:
        neighbours_cells = []
        neighbours_cells.append(trips_by_regions[trips_by_regions['region']==j-row-1][cur_time_column].values)
        neighbours_cells.append(trips_by_regions[trips_by_regions['region']==j-row][cur_time_column].values)
        neighbours_cells.append(trips_by_regions[trips_by_regions['region']==j-1][cur_time_column].values)
        neighbours_cells.append(trips_by_regions[trips_by_regions['region']==j+1][cur_time_column].values)
        neighbours_cells.append(trips_by_regions[trips_by_regions['region']==j+row-1][cur_time_column].values)
        neighbours_cells.append(trips_by_regions[trips_by_regions['region']==j+row][cur_time_column].values)
        neighbours_cells.append(trips_by_regions[trips_by_regions['region']==j+row+1][cur_time_column].values)
        region_mean.append(np.mean(neighbours_cells))
    if i%24==0:
        print(cur_datetime)
    trips_by_regions_neighbours[cur_time_column] = region_mean
trips_by_regions_neighbours.to_csv('trips_by_regions_neighbours_{}.csv'.format(number), index=False)

2016-06-01 00:00:00
2016-06-02 00:00:00
2016-06-03 00:00:00
2016-06-04 00:00:00
2016-06-05 00:00:00
2016-06-06 00:00:00
2016-06-07 00:00:00
2016-06-08 00:00:00
2016-06-09 00:00:00
2016-06-10 00:00:00
2016-06-11 00:00:00
2016-06-12 00:00:00
2016-06-13 00:00:00
2016-06-14 00:00:00
2016-06-15 00:00:00
2016-06-16 00:00:00
2016-06-17 00:00:00
2016-06-18 00:00:00
2016-06-19 00:00:00
2016-06-20 00:00:00
2016-06-21 00:00:00
2016-06-22 00:00:00
2016-06-23 00:00:00
2016-06-24 00:00:00
2016-06-25 00:00:00
2016-06-26 00:00:00
2016-06-27 00:00:00
2016-06-28 00:00:00
2016-06-29 00:00:00
2016-06-30 00:00:00
Wall time: 5min 2s
